In [7]:
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


class LabProtocolStep(BaseModel):
    thinking: str = Field(
        ...,
        description="A brief explanation of the reasoning or intent behind this step; what is being accomplished or considered."
    )
    instruction: str = Field(
        ...,
        description="A clear, actionable instruction for what to do in this step; should be concise and unambiguous."
    )
    materials: list[str] = Field(
        default_factory=list,
        description="A list of materials, reagents, or equipment required specifically for this step; leave empty if none."
    )
    notes: str = Field(
        default="",
        description="Any additional context, tips, or warnings relevant to this step; optional."
    )

class LabProtocol(BaseModel):
    thinking: str = Field(
        ...,
        description="A high-level summary of the overall reasoning, purpose, or approach of the protocol as a whole."
    )
    steps: list[LabProtocolStep] = Field(
        ...,
        description="An ordered list of LabProtocolStep objects representing each step in the protocol."
    )

client = genai.Client(api_key=os.getenv("GOOGLE_GENERATIVE_AI_API_KEY"))

video_file_name = "output.mp4"
video_bytes = open(video_file_name, 'rb').read()

content = types.Content(
        parts=[
            types.Part(
                inline_data=types.Blob(data=video_bytes, mime_type='video/mp4')
            ),
            types.Part(text='break this down frame by frame and make a lab protocol for it')
        ]
    )

In [8]:
response = client.models.generate_content(
    model="models/gemini-2.5-pro",
    contents=content,
    config={
        "response_mime_type": "application/json",
        "response_schema": LabProtocol,
    },
)

ServerError: 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}

In [6]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='{\n  "thinking": "The user wants a lab protocol for the concept of \'breaking something down frame by frame\'. I will interpret this as a procedure for analyzing a digital video file on a frame-by-frame basis, a common task in scientific research (e.g., motion analysis, microscopy, behavioral studies). The protocol will guide the user from software setup and defining parameters through the systematic analysis loop to data validation and archiving, ensuring a reproducible and rigorous workflow.",\n  "steps": [\n    {\n      "thinking": "The first step is to prepare the necessary tools. This involves selecting and setting up the software and loading the media file that will be analyzed. This ensures the workspace is ready bef

In [ ]:
print(response.usage_metadata)

In [2]:
from google import genai
import time

client = genai.Client(api_key=os.getenv("GOOGLE_GENERATIVE_AI_API_KEY"))
prompt = "Tell me about this video"
your_file = client.files.upload(file="output.mov")

print(
    client.models.count_tokens(
        model="gemini-2.5-pro", contents=content
    )
)
# ( e.g., total_tokens: 300 )

print(response.usage_metadata)
# ( e.g., prompt_token_count: 301, candidates_token_count: 60, total_token_count: 361 )

total_tokens=18870 cached_content_token_count=None


NameError: name 'response' is not defined

In [ ]:
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_videos: list[Video] = response.parsed

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=MY_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)

stream = client.chat.completions.create(
    model="gemini-2.5-pro",
    n=1,
    messages=[
        {
            "role": "user",
            "content": "Summarize the video"
        }
    ],
    stream=True,
    stream_options={'include_usage': True},
    extra_body={
        'extra_body':
        {
            'google': {
              'cached_content': "cachedContents/0000aaaa1111bbbb2222cccc3333dddd4444eeee"
          }
        }
    }
)

for chunk in stream:
    print(chunk)
    print(chunk.usage.to_dict())